<a href="https://colab.research.google.com/github/Renato-55/UtadinosHackerton/blob/main/Hackton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Renato-55/UtadinosHackerton

In [ ]:
!pip install tensorflow

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

#path to data
#path = "/content/UtadinosHackerton/data_cy.csv"
path = "/content/UtadinosHackerton/CO2 Emissions_Canada.csv"
#path = "C://Users//renat//Desktop//Hackton//Ficheiros//data_cy.csv"
#path = "C://Users//renat//Desktop//Hackton//Ficheiros//data_pt_esp.csv"

#Load Data
df = pd.read_csv(path)

In [9]:
#data cleaning
from sklearn.utils import shuffle
df = shuffle(df)
#target
y_features = ["CO2 Emissions(g/km)"]

y = df[y_features]

In [ ]:
null_val = df.isnull().sum().sum()

if null_val == 0:
  print("No null values")
else:
  print(f"Null values: {null_val}")

df.fillna('0',inplace=True)

data_num_features = df.select_dtypes(include=np.number)

data_num_features = data_num_features.drop(y_features,axis = 1)

In [11]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model():
  model = keras.Sequential([
      layers.Dense(data_num_features.shape[0], activation="relu"),
      layers.Dense(64, activation="relu"),
      layers.Dense(64, activation="relu"),
      layers.Dense(1)
  ])
  model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
  return model

In [ ]:
import numpy as np
k = 5 
num_val_samples = len(data_num_features) // k 
num_epochs = 40

Fold = int(y.shape[0]/k)

#divide a data em train_Data e eval_data
X_train = data_num_features[:Fold*(k-1)-1]
y_train = y[:Fold*(k-1)-1]
X_eval = data_num_features[Fold*(k-1)+1:Fold*(k)]
y_eval = y[Fold*(k-1)+1:Fold*(k)]

model = build_model()

callbacks = [
    keras.callbacks.ModelCheckpoint("callback_best_co2_model.keras",
                                    save_best_only=True, monitor='loss')
]

model.fit(X_train, y_train, epochs=num_epochs, batch_size=41, callbacks=callbacks)

model = keras.models.load_model("callback_best_co2_model.keras")

In [ ]:
val_mse, val_mae = model.evaluate(X_eval, y_eval)

In [ ]:
print(f"Fold: {Fold}")
print(f"y shape: {y.shape[0]}")
print(f"k : {y.shape[0]/Fold}")
print(f"k train end : {Fold*(k-1)-1}")
print(f"k eval init : {Fold*(k-1)+1}")
print(f"k eval end : {Fold*(k)}")


In [ ]:
print(f"Mae {val_mae}")
print(f"Mean {y_eval.mean()}")
print(f"Error {val_mae/y_eval.mean()}")
print(f"Accuracy {100 - val_mae/y_eval.mean()}")


In [ ]:
print(f"Numeric data shape : {data_num_features.shape}")
print(f"All data shape: {df.shape}")

for x in data_num_features:
  print(x)

In [ ]:
print(model.predict(X_eval.tail()))

print(y.tail())

In [ ]:
tf.saved_model.save(model,"/content/savedModel")

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler, 
                                   PolynomialFeatures)

mlr = LinearRegression()

mlr.fit(X_train, y_train)

val_mse_linear, val_mae_linear = model.evaluate(X_eval, y_eval)

In [ ]:
print(f"Mae {val_mae_linear}")
print(f"Mean {y_eval.mean()}")
print(f"Error {val_mae_linear/y_eval.mean()}")
print(f"Accuracy {100 - val_mae_linear/y_eval.mean()}")

In [ ]:
from tensorflow import keras
model = keras.models.load_model("callback_best_co2_model.keras")

mean_emission = np.mean(y)

class Carro:

  def __init__(self):
    self.marca = "Renault"
    self.modelo = "Megane"
    self.tamMotor = 2
    self.numCilindros = 4
    self.tipoCombustivel = "Gasoleo"
    self.mediaMotorCity = 7
    self.mediaMotorHwy = 6
    self.mediaMotorComb = self.mediaMotorCity*0.55 + self.mediaMotorCity*0.45
    self.mediaMotorCombMpg = self.mediaMotorComb*6.534
    #valor target default
  
  def lerDados(self):
    self.marca = input("Type the car Make: ")
    self.modelo = input("Type the car Model: ")
    self.tamMotor = int(input("Type the motor size: "))
    self.numCilindros = int(input("Type the num of cilinders: "))
    self.tipoCombustivel = input("Type the type of fuel (X - Gasoline) (Z - diesel)(D- eletric)")
    self.mediaMotorCity=float(input("Type the avarege consumption of fuel driving in the city per 100km :"))
    self.mediaMotorHwy=float(input("Type the avarege consumption of fuel driving in the highway per 100km :"))
    self.mediaMotorComb = self.mediaMotorCity*0.55 + self.mediaMotorCity*0.45
    self.mediaMotorCombMpg = self.mediaMotorComb*6.534
  
  def predict(self,model):  
    #one = np.array( [  self.numCilindros,  self.mediaMotorCity,self.mediaMotorHwy,
    #                   self.mediaMotorComb,self.mediaMotorCombMpg, self.consumoCombinado])

    one = tf.constant( [[  self.tamMotor, self.numCilindros,  self.mediaMotorCity,self.mediaMotorHwy,
                       self.mediaMotorComb,self.mediaMotorCombMpg]])
    
    predicted_emission = model.predict(one)
    print(f"Is Expected to the car {self.marca} has an co2 (g/km) emission of {predicted_emission}\n")
    
    
    dif = predicted_emission- y.mean()[0] 
    if(dif > 0):
      print(f"Compared to the known models, your car is {abs(dif)} c02 (g/km) horse.\n")
    elif(dif <= 0):
      print(f"Compared to the known models, your car is {abs(dif)} c02 (g/km) better.\n")
    
    

c = Carro()
#Carro.lerDados(c)
print("Neural Network Model")
Carro.predict(c,model)
print("\nLinear Regression Model")
Carro.predict(c,mlr)

